In [1]:
# note that this custom dataset is not prepared on the top of geometric Dataset(pytorch's inbuilt)
import os
import torch
import glob
import numpy as np 
import random
import math
from os import listdir
from os.path import isfile, join

processed_dir = "./PPI_GNN-main/Human_features"
npy_file = "./PPI_GNN-main/Human_features/npy_file_new(human_dataset).npy"
npy_ar = np.load(npy_file)
print("SHAPE:", npy_ar.shape)
print(npy_ar)


from torch.utils.data import Dataset as Dataset_n
from torch_geometric.data import DataLoader as DataLoader_n

class LabelledDataset(Dataset_n):
    def __init__(self, npy_file, processed_dir):
      self.npy_ar = np.load(npy_file)
      self.processed_dir = processed_dir
      self.protein_1 = self.npy_ar[:,2]
      self.protein_2 = self.npy_ar[:,5]
      self.label = self.npy_ar[:,6].astype(float)
      self.n_samples = self.npy_ar.shape[0]

    def __len__(self):
      return(self.n_samples)

    def __getitem__(self, index):
      prot_1 = os.path.join(self.processed_dir, self.protein_1[index]+".pt")
      prot_2 = os.path.join(self.processed_dir, self.protein_2[index]+".pt")
      #print(f'Second prot is {prot_2}')
      prot_1 = torch.load(glob.glob(prot_1)[0])
      #print(f'Here lies {glob.glob(prot_2)}')
      prot_2 = torch.load(glob.glob(prot_2)[0])
      return prot_1, prot_2, torch.tensor(self.label[index])



dataset = LabelledDataset(npy_file = npy_file ,processed_dir=processed_dir)

final_pairs =  np.load(npy_file)
size = final_pairs.shape[0]
print("\nSize:", size)
seed = 42
torch.manual_seed(seed)
#print(math.floor(0.8 * size))
#Make iterables using dataloader class  
trainset, testset = torch.utils.data.random_split(dataset, [math.floor(0.8 * size), size - math.floor(0.8 * size) ])
#print(trainset[0])
trainloader = DataLoader_n(dataset= trainset, batch_size= 4, num_workers = 0)
testloader = DataLoader_n(dataset= testset, batch_size= 4, num_workers = 0)
print("\nLength:")
print(len(trainloader))
print(len(testloader))

SHAPE: (22217, 7)
[['NP_663777' 'Q13114' '1FLK' ... 'P26842' '5TL5' '1']
 ['NP_003630' 'Q9Y6K9' '2JVX' ... 'Q14790' '1F9E' '1']
 ['NP_003001' 'P45985' '3ALN' ... 'Q12852' '5CEN' '1']
 ...
 ['NP_000779' 'P27707' '1P5Z' ... 'P00338' '1I10' '0']
 ['NP_008828' 'P31274' '2LP0' ... 'Q9H257' '6E25' '0']
 ['NP_001740' 'P04632' '1KFU' ... 'Q15052' '1UJY' '1']]

Size: 22217

Length:
4444
1111


c:\Users\laura\anaconda3\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [2]:
import pandas as pd

# Assuming final_pairs is a NumPy array containing the data
df = pd.DataFrame(final_pairs)
df

,0,1,2,3,4,5,6
0,NP_663777,Q13114,1FLK,NP_001233,P26842,5TL5,1
1,NP_003630,Q9Y6K9,2JVX,NP_001073594,Q14790,1F9E,1
2,NP_003001,P45985,3ALN,NP_006292,Q12852,5CEN,1
3,NP_060901,Q9P104,1J0W,NP_000199,P06213,1GAG,1
4,NP_001020276,P48729,5FQD,NP_001895,P35222,1G3J,1
...,...,...,...,...,...,...,...
22212,NP_000240,P40692,3RBN,NP_000404,P14061,1A27,0
22213,P10619,P10619,1IVY,NP_005989,P49368,6NR8,0
22214,NP_000779,P27707,1P5Z,NP_005557,P00338,1I10,0
22215,NP_008828,P31274,2LP0,NP_434700,Q9H257,6E25,0


In [20]:
col0 = sorted(df[0].unique())
print("Length col 0: ", len(col0))

col1 = sorted(df[1].unique())
print("Length col 1: ", len(col1))

col3 = sorted(df[3].unique())
print("Length col 3: ", len(col3))

col4 = sorted(df[4].unique())
print("Length col 4: ", len(col4))

for (x1,x2,x3,x4) in zip(col0, col1, col3, col4):
    print(x1, x2, x3, x4, sep="\t\t")


Length col 0:  3615
Length col 1:  3527
Length col 3:  3327
Length col 4:  3273
NP_000005		A0A0C4DGV4		NP_000005		A0A0C4DGV4
NP_000011		A0A173ADG5		NP_000006		A0A173ADG5
NP_000012		A1A4S6		NP_000011		A7KAX9
NP_000015		A5YKK6		NP_000012		A8K8P3
NP_000016		A7KAX9		NP_000013		O00139
NP_000018		A8K8P3		NP_000015		O00141
NP_000020		A8MT69		NP_000016		O00151
NP_000021		D3DT19		NP_000018		O00154
NP_000023		O00141		NP_000020		O00161
NP_000029		O00151		NP_000023		O00168
NP_000030		O00161		NP_000025		O00170
NP_000032		O00168		NP_000026		O00187
NP_000033		O00170		NP_000029		O00189
NP_000034		O00180		NP_000030		O00198
NP_000035		O00187		NP_000032		O00206
NP_000036		O00189		NP_000033		O00212
NP_000039		O00194		NP_000034		O00213
NP_000042		O00198		NP_000035		O00214
NP_000043		O00204		NP_000036		O00220
NP_000044		O00206		NP_000042		O00231
NP_000045		O00212		NP_000043		O00232
NP_000047		O00213		NP_000045		O00238
NP_000048		O00214		NP_000046		O00241
NP_000050		O00220		NP_000048		O00244
NP_000052		O0022

: 

In [6]:
# from bio_embeddings.embed import ProtTransBertBFDEmbedder, SeqVecEmbedder
# # import numpy as np
# # import torch 

# seq = 'MVTYDFGSDEMHD' # A protein sequence of length L

# embedder = SeqVecEmbedder()
# embedding = embedder.embed(seq)
# protein_embd = torch.tensor(embedding).sum(dim=0) # Vector with shape [L x 1024]
# np_arr = protein_embd.cpu().detach().numpy()